In [2]:
####################################################################################################
########################### 1 - IMPORT DAS BIBLIOTECAS ############################################# 
####################################################################################################

import http.client
import json
import pandas as pd
from datetime import datetime
import numpy as np
import time

from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [3]:
####################################################################################################
#################################### 2 - FUNÇÕES ################################################### 
####################################################################################################

def datetime_handler(x):
    if isinstance(x, pd.Timestamp):
        return x.isoformat()
    raise TypeError("Unknown type")
    
def read_sheets(sheets):
    result = sheet.values().get(spreadsheetId='1P1LbVDyHmyebOUMIa-wyjyJ0fChiHh92p6Hj1FPPItE',
                            range=sheets).execute()
    values_old = result.get('values', [])
    return values_old


def insert_sheets(df, sheet):
    body = {
    'values': df
    }
    result = service.spreadsheets().values().update(
        spreadsheetId='1P1LbVDyHmyebOUMIa-wyjyJ0fChiHh92p6Hj1FPPItE', range= sheet,
        valueInputOption="USER_ENTERED", body=body).execute()

In [4]:
####################################################################################################
############################## 3 - API E SERVIÇOS GOOGLE ########################################### 
####################################################################################################

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

In [ ]:
dePara = read_sheets('deParaCustomField1!C2:C')
df_dePara = pd.DataFrame(data = dePara)
df_dePara = df_dePara.rename(columns = {0: 'idDatasource'})
lista_dePara = df_dePara.values.tolist()

print(lista_dePara)

In [ ]:
####################################################################################################
################################## 5 - ETL E INSERT ################################################ 
####################################################################################################

for x in range(0, len(lista_dePara)):

    conn = http.client.HTTPSConnection("api.pandape.com.br")
    payload = ''
    headers = {
      'Authorization': 'Bearer '
    }
    conn.request("GET", f"/v2/data-sources/items?IdDatasource={lista_dePara[x][0]}&Page=1&PageSize=1000", payload, headers)
    res = conn.getresponse()
    data = res.read()

    data_dict = json.loads(data)

    data_json = json.dumps(data_dict['items'], default = datetime_handler)
    data_dict2 = json.loads(data_json)


    df_Datasource = pd.DataFrame(data = data_dict2)

    df_Datasource['IdDatasource'] = lista_dePara[x][0]

    df_Datasource = df_Datasource[["IdDatasource", "idDatasourceItem", "text"]]

    display(df_Datasource)

    list_Datasource = df_Datasource.values.tolist()

    tamanho = len(read_sheets('Datasource'))

    insert_sheets(list_Datasource, f'Datasource!A{tamanho + 1}')


,IdDatasource,idDatasourceItem,text
0,9969,7014,Colaborador será promovido
1,9969,7015,Colaborador será transferido para outro posto
2,9969,7016,Colaborador será demitido
3,9969,7017,Colaborador pediu demissão
4,9969,7018,Afastamento INSS
5,9969,7019,Licença Maternidade


,IdDatasource,idDatasourceItem,text
0,9968,7011,G.Eletro
1,9968,7012,Segurança
2,9968,7013,Serviços


,IdDatasource,idDatasourceItem,text
0,9967,6997,Ceará
1,9967,6998,Piauí
2,9967,6999,Rio de Janeiro
3,9967,7000,Pernambuco
4,9967,7001,Pará
5,9967,7002,Minas Gerais
6,9967,7003,Maranhão
7,9967,7004,G. Eletro
8,9967,7005,Frei Caneca
9,9967,7006,Espírito Santo


,IdDatasource,idDatasourceItem,text
0,9966,6977,Presidência
1,9966,6978,Operações
2,9966,6979,Treinamento
3,9966,6980,TI
4,9966,6981,Suprimentos
5,9966,6982,S.A.
6,9966,6983,Marketing
7,9966,6984,Jurídico
8,9966,6985,Implementação
9,9966,6986,Gestão de Processos


,IdDatasource,idDatasourceItem,text
0,9965,6962,Operações SP - Zona Sul
1,9965,6963,Operações SP - Zona Oeste
2,9965,6964,Operações SP - Zona Norte
3,9965,6965,Operações SP - Zona Leste
4,9965,6966,Operações SP - Vila Olimpia
5,9965,6967,Operações SP - Sudoeste
6,9965,6968,Operações SP - Sudeste
7,9965,6969,Operações SP - Interior II
8,9965,6970,Operações SP - Interior I
9,9965,6971,Operações SP - Interior


...

,IdDatasource,idDatasourceItem,text
0,9963,6956,Indiferente
1,9963,6957,Masculino
2,9963,6958,Feminino


,IdDatasource,idDatasourceItem,text
0,9962,6953,Outros
1,9962,6954,Noturno
2,9962,6955,Diurno


,IdDatasource,idDatasourceItem,text
0,9961,6945,Outros
1,9961,6946,4 x 2
2,9961,6947,5 x 2
3,9961,6948,6 x 2
4,9961,6949,6 x 1
5,9961,6950,12 x 36
6,9961,6951,12 x 36 - Par
7,9961,6952,12 x 36 - Ímpar


,IdDatasource,idDatasourceItem,text
0,9960,6933,Outros
1,9960,6934,22h às 10h
2,9960,6935,19h às 7h
3,9960,6936,18h às 6h
4,9960,6937,15h30 às 22h
5,9960,6938,10h às 22h
6,9960,6939,8h às 18h
7,9960,6940,7h às 19h
8,9960,6941,7h às 15h30
9,9960,6942,7h às 15h20


,IdDatasource,idDatasourceItem,text
0,9959,6924,Hotelaria
1,9959,6925,Stands e Obras
2,9959,6926,Indústria
3,9959,6927,Instituições de Ensino
4,9959,6928,Administrativo
5,9959,6929,Hospitalar
6,9959,6930,Shopping
7,9959,6931,Corporativo
8,9959,6932,Imobiliário
